In [ ]:
!pip install streamlit
!pip install dash

In [ ]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st

Time Period & 3 country comparison


In [ ]:
# Time Period & 3 country comparison

import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

# Load the CSV file and save as DataFrame
file_id = '1-nKxWsR4Yc8cI7W3jeYwh0Ef1zq9tvIQ'
file_path = f'https://drive.google.com/uc?id={file_id}'
df_import_export = pd.read_csv(file_path)
df_import_export['Date'] = pd.to_datetime(df_import_export['Date'], dayfirst=True)
df_import_export['Year'] = df_import_export['Date'].dt.year
df_import_export['Month'] = df_import_export['Date'].dt.month
df_import_export['Day'] = df_import_export['Date'].dt.day

# Layout of the app
app.layout = html.Div([
    html.H1("Import/Export Dataset Dashboard"),

    # Dropdown for selecting Import or Export
    dcc.Dropdown(
        id='import-export-dropdown',
        options=[
            {'label': 'Import', 'value': 'Import'},
            {'label': 'Export', 'value': 'Export'}
        ],
        multi=True,
        placeholder="Select Import or Export"
    ),

    # Dropdown for selecting Year, Month, or Day
    dcc.Dropdown(
        id='time-period-dropdown',
        options=[
            {'label': 'Year', 'value': 'Year'},
            {'label': 'Month', 'value': 'Month'},
            {'label': 'Day', 'value': 'Day'}
        ],
        placeholder="Select Time Period"
    ),

    # Dropdown for selecting Quantity or Value
    dcc.Dropdown(
        id='quantity-value-dropdown',
        options=[
            {'label': 'Quantity', 'value': 'Quantity'},
            {'label': 'Value', 'value': 'Value'}
        ],
        placeholder="Select Quantity or Value"
    ),

    # Graph to display the selected data
    dcc.Graph(id='import-export-graph'),

    # Separator
    html.Hr(),

    html.H1("Country Comparison Dashboard"),

    # Country selection dropdown
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in df_import_export['Country'].unique()],
        multi=True,
        placeholder="Select up to 3 countries for comparison"
    ),

    # Import/Export selection dropdown
    dcc.Dropdown(
        id='country-import-export-dropdown',
        options=[
            {'label': 'Import', 'value': 'Import'},
            {'label': 'Export', 'value': 'Export'},
            {'label': 'Both', 'value': 'Both'}
        ],
        placeholder="Select Import, Export, or Both"
    ),

    # Graph placeholders for country comparison
    dcc.Graph(id='import-export-country-count'),
    dcc.Graph(id='category-count-country'),
    dcc.Graph(id='shipping-method-count-country')
])

# Callback to update the import/export graph
@app.callback(
    Output('import-export-graph', 'figure'),
    [Input('import-export-dropdown', 'value'),
     Input('time-period-dropdown', 'value'),
     Input('quantity-value-dropdown', 'value')]
)
def update_graph(import_export_selection, time_period, quantity_value):
    if not import_export_selection or not time_period or not quantity_value:
        return {}

    # Create a column for the x-axis based on the selected time period
    x_axis = time_period

    # Filter for import/export
    filtered_df = df_import_export[df_import_export['Import_Export'].isin(import_export_selection)]

    # Group by the selected time period and calculate the sum of the quantity/value
    grouped_df = filtered_df.groupby([x_axis, 'Import_Export'], as_index=False).agg({quantity_value: 'sum'})

    # Create the bar chart
    fig = px.bar(
        grouped_df,
        x=x_axis,
        y=quantity_value,
        color='Import_Export',
        title=f"{quantity_value} by {time_period}",
        labels={quantity_value: quantity_value, x_axis: time_period},
        color_discrete_map={
            'Import': 'blue',
            'Export': 'orange'
        }
    )

    return fig

# Callback to update the country comparison charts
@app.callback(
    [
        Output('import-export-country-count', 'figure'),
        Output('category-count-country', 'figure'),
        Output('shipping-method-count-country', 'figure')
    ],
    [
        Input('country-dropdown', 'value'),
        Input('country-import-export-dropdown', 'value')
    ]
)
def update_charts(selected_countries, selected_import_export):
    if not selected_countries or not selected_import_export:
        return {}, {}, {}

    # Limit the selection to 3 countries
    if len(selected_countries) > 3:
        selected_countries = selected_countries[:3]

    # Filter data based on selected countries and import/export option
    if selected_import_export != 'Both':
        filtered_df = df_import_export[
            (df_import_export['Country'].isin(selected_countries)) &
            (df_import_export['Import_Export'] == selected_import_export)
        ]
    else:
        filtered_df = df_import_export[df_import_export['Country'].isin(selected_countries)]

    # Import/Export Count & Port Count by Country
    import_export_fig = px.bar(
        filtered_df.groupby(['Country', 'Import_Export']).size().reset_index(name='Count'),
        x='Country', y='Count', color='Import_Export',
        title='Import/Export Count by Country',
        barmode='group',
        color_discrete_sequence=['blue', 'green']
    )

    # Count of Categories by Country
    category_fig = px.bar(
        filtered_df.groupby(['Country', 'Category']).size().reset_index(name='Count'),
        x='Country', y='Count', color='Category',
        title='Count of Categories by Country',
        barmode='group',
        color_discrete_sequence=px.colors.qualitative.Plotly
    )

    # Count of Shipping Methods by Country
    shipping_method_fig = px.bar(
        filtered_df.groupby(['Country', 'Shipping_Method']).size().reset_index(name='Count'),
        x='Country', y='Count', color='Shipping_Method',
        title='Count of Shipping Methods by Country',
        barmode='group',
        color_discrete_sequence=px.colors.qualitative.Plotly
    )

    return import_export_fig, category_fig, shipping_method_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

# Ports comparison & Ports Table


In [ ]:
# Ports comparison & Ports Table

import dash
from dash import dcc, html, Input, Output, dash_table
import plotly.express as px
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

# Load the CSV file from Google Drive and prepare DataFrame
file_id = '1-nKxWsR4Yc8cI7W3jeYwh0Ef1zq9tvIQ'
file_path = f'https://drive.google.com/uc?id={file_id}'
df_import_export = pd.read_csv(file_path)
df_import_export['Date'] = pd.to_datetime(df_import_export['Date'], dayfirst=True)
df_import_export['Year'] = df_import_export['Date'].dt.year
df_import_export['Month'] = df_import_export['Date'].dt.month
df_import_export['Day'] = df_import_export['Date'].dt.day

# Layout of the app
app.layout = html.Div([
    html.H1("Import/Export Dashboard"),

    # Country Dropdown for Ports Comparison Chart
    html.Div([
        html.Label("Country for Ports Comparison"),
        dcc.Dropdown(
            id='country-dropdown-chart',
            options=[{'label': country, 'value': country} for country in df_import_export['Country'].unique()],
            placeholder="Select a country for chart"
        ),
    ]),

    # Dropdown for selecting ports for comparison (for chart)
    dcc.Dropdown(
        id='port-dropdown',
        multi=True,
        placeholder="Select up to 2 ports for comparison"
    ),

    # Ports Comparison Chart
    dcc.Graph(id='import-export-port-bar-chart'),

    # Pie Charts for Shipping Methods and Payment Terms
    html.Div([
        dcc.Graph(id='shipping-methods-pie-chart'),
        dcc.Graph(id='payment-terms-pie-chart')
    ], style={'display': 'flex', 'justify-content': 'space-around'}),

    # Country Dropdown for Ports Table
    html.Div([
        html.Label("Country for Ports Table"),
        dcc.Dropdown(
            id='country-dropdown-table',
            options=[{'label': country, 'value': country} for country in df_import_export['Country'].unique()],
            placeholder="Select a country for table"
        ),
    ]),

    # Filters for Ports Table
    dcc.Dropdown(
        id='import-export-dropdown',
        options=[
            {'label': 'Import', 'value': 'Import'},
            {'label': 'Export', 'value': 'Export'},
            {'label': 'Both', 'value': 'Both'}
        ],
        placeholder="Select Import, Export, or Both",
        clearable=True
    ),
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': category, 'value': category} for category in df_import_export['Category'].unique()],
        placeholder="Select Category",
        clearable=True
    ),
    dcc.Dropdown(
        id='shipping-method-dropdown',
        options=[{'label': method, 'value': method} for method in df_import_export['Shipping_Method'].unique()],
        placeholder="Select Shipping Method",
        clearable=True
    ),
    dcc.Dropdown(
        id='payment-terms-dropdown',
        options=[{'label': term, 'value': term} for term in df_import_export['Payment_Terms'].unique()],
        placeholder="Select Payment Terms",
        clearable=True
    ),

    # Table for displaying available ports with average quantity and value
    dash_table.DataTable(
        id='ports-table',
        columns=[
            {'name': 'Port', 'id': 'Port'},
            {'name': 'Average Quantity', 'id': 'Average_Quantity'},
            {'name': 'Average Value', 'id': 'Average_Value'}
        ],
        page_size=10,
        style_table={'overflowX': 'auto'},
    )
])

# Callback to update the port dropdown based on the selected country for chart
@app.callback(
    Output('port-dropdown', 'options'),
    Input('country-dropdown-chart', 'value')
)
def update_port_dropdown(selected_country):
    if not selected_country:
        return []

    # Filter ports based on selected country
    filtered_df = df_import_export[df_import_export['Country'] == selected_country]
    port_options = [{'label': port, 'value': port} for port in filtered_df['Port'].unique()]
    return port_options

# Callback to update the bar chart based on selected ports
@app.callback(
    Output('import-export-port-bar-chart', 'figure'),
    Input('country-dropdown-chart', 'value'),
    Input('port-dropdown', 'value')
)
def update_import_export_port_chart(selected_country, selected_ports):
    if not selected_country or not selected_ports or len(selected_ports) > 2:
        return {}

    # Filter data for selected country and ports
    filtered_df = df_import_export[(df_import_export['Country'] == selected_country) &
                                   (df_import_export['Port'].isin(selected_ports))]

    # Create a bar chart
    fig = px.bar(
        filtered_df,
        x='Port',
        y='Value',
        color='Import_Export',
        title='Import and Export Values by Port',
        barmode='group',
        color_discrete_sequence=['blue', 'green']
    )
    return fig

# Callback to update pie charts for shipping methods
@app.callback(
    Output('shipping-methods-pie-chart', 'figure'),
    Input('country-dropdown-chart', 'value'),
    Input('port-dropdown', 'value')
)
def update_shipping_methods_pie_chart(selected_country, selected_ports):
    if not selected_country or not selected_ports or len(selected_ports) > 2:
        return {}

    # Filter data for selected country and ports
    filtered_df = df_import_export[(df_import_export['Country'] == selected_country) &
                                   (df_import_export['Port'].isin(selected_ports))]

    # Create pie chart for shipping methods
    fig = px.pie(
        filtered_df,
        names='Shipping_Method',
        values='Value',
        title=f'Shipping Methods Distribution for Ports: {", ".join(selected_ports)}',
        color_discrete_sequence=px.colors.qualitative.Plotly
    )
    return fig

# Callback to update pie charts for payment terms
@app.callback(
    Output('payment-terms-pie-chart', 'figure'),
    Input('country-dropdown-chart', 'value'),
    Input('port-dropdown', 'value')
)
def update_payment_terms_pie_chart(selected_country, selected_ports):
    if not selected_country or not selected_ports or len(selected_ports) > 2:
        return {}

    # Filter data for selected country and ports
    filtered_df = df_import_export[(df_import_export['Country'] == selected_country) &
                                   (df_import_export['Port'].isin(selected_ports))]

    # Create pie chart for payment terms
    fig = px.pie(
        filtered_df,
        names='Payment_Terms',
        values='Value',
        title=f'Payment Terms Distribution for Ports: {", ".join(selected_ports)}',
        color_discrete_sequence=px.colors.qualitative.Plotly
    )
    return fig

# Callback to update the ports table based on selected filters
@app.callback(
    Output('ports-table', 'data'),
    [
        Input('country-dropdown-table', 'value'),
        Input('import-export-dropdown', 'value'),
        Input('category-dropdown', 'value'),
        Input('shipping-method-dropdown', 'value'),
        Input('payment-terms-dropdown', 'value')
    ]
)
def update_ports_table(selected_country, selected_import_export, selected_category, selected_shipping_method, selected_payment_terms):
    # Start with the initial DataFrame
    filtered_df = df_import_export

    # Filter by country
    if selected_country:
        filtered_df = filtered_df[filtered_df['Country'] == selected_country]

    # Filter by Import/Export
    if selected_import_export and selected_import_export != 'Both':
        filtered_df = filtered_df[filtered_df['Import_Export'] == selected_import_export]

    # Filter by Category
    if selected_category:
        filtered_df = filtered_df[filtered_df['Category'] == selected_category]

    # Filter by Shipping Method
    if selected_shipping_method:
        filtered_df = filtered_df[filtered_df['Shipping_Method'] == selected_shipping_method]

    # Filter by Payment Terms
    if selected_payment_terms:
        filtered_df = filtered_df[filtered_df['Payment_Terms'] == selected_payment_terms]

    # Group by Port and calculate average Quantity and Value
    if not filtered_df.empty:
        ports_data = filtered_df.groupby('Port').agg(
            Average_Quantity=('Quantity', 'mean'),
            Average_Value=('Value', 'mean')
        ).reset_index()
    else:
        ports_data = pd.DataFrame(columns=['Port', 'Average_Quantity', 'Average_Value'])

    return ports_data.to_dict('records')

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

In [ ]:
######## ENTIRE DASHBOARD

import dash
from dash import dcc, html, Input, Output, dash_table
import plotly.express as px
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

# Load the CSV file and save as DataFrame
file_id = '1-nKxWsR4Yc8cI7W3jeYwh0Ef1zq9tvIQ'
file_path = f'https://drive.google.com/uc?id={file_id}'
df_import_export = pd.read_csv(file_path)
df_import_export['Date'] = pd.to_datetime(df_import_export['Date'], dayfirst=True)
df_import_export['Year'] = df_import_export['Date'].dt.year
df_import_export['Month'] = df_import_export['Date'].dt.month
df_import_export['Day'] = df_import_export['Date'].dt.day

# Layout of the app
app.layout = html.Div([
    html.H1("Import/Export Dataset Dashboard"),

    # Dropdowns for time period and import/export type selection
    dcc.Dropdown(
        id='import-export-dropdown',
        options=[
            {'label': 'Import', 'value': 'Import'},
            {'label': 'Export', 'value': 'Export'}
        ],
        multi=True,
        placeholder="Select Import or Export"
    ),
    dcc.Dropdown(
        id='time-period-dropdown',
        options=[
            {'label': 'Year', 'value': 'Year'},
            {'label': 'Month', 'value': 'Month'},
            {'label': 'Day', 'value': 'Day'}
        ],
        placeholder="Select Time Period"
    ),
    dcc.Dropdown(
        id='quantity-value-dropdown',
        options=[
            {'label': 'Quantity', 'value': 'Quantity'},
            {'label': 'Value', 'value': 'Value'}
        ],
        placeholder="Select Quantity or Value"
    ),
    dcc.Graph(id='import-export-graph'),

    # Country comparison section
    html.Hr(),
    html.H1("Country Comparison Dashboard"),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in df_import_export['Country'].unique()],
        multi=True,
        placeholder="Select up to 3 countries for comparison"
    ),
    dcc.Dropdown(
        id='country-import-export-dropdown',
        options=[
            {'label': 'Import', 'value': 'Import'},
            {'label': 'Export', 'value': 'Export'},
            {'label': 'Both', 'value': 'Both'}
        ],
        placeholder="Select Import, Export, or Both"
    ),
    dcc.Graph(id='import-export-country-count'),
    dcc.Graph(id='category-count-country'),
    dcc.Graph(id='shipping-method-count-country'),

    # Ports comparison chart
    html.Hr(),
    html.H1("Ports Comparison Dashboard"),
    html.Label("Country for Ports Comparison"),
    dcc.Dropdown(
        id='country-dropdown-chart',
        options=[{'label': country, 'value': country} for country in df_import_export['Country'].unique()],
        placeholder="Select a country for chart"
    ),
    dcc.Dropdown(id='port-dropdown', multi=True, placeholder="Select up to 2 ports for comparison"),
    dcc.Graph(id='import-export-port-bar-chart'),
    dcc.Graph(id='shipping-methods-pie-chart'),
    dcc.Graph(id='payment-terms-pie-chart'),

    # Ports table section
    html.Hr(),
    html.H1("Ports Table"),
    html.Label("Country for Ports Table"),
    dcc.Dropdown(
        id='country-dropdown-table',
        options=[{'label': country, 'value': country} for country in df_import_export['Country'].unique()],
        placeholder="Select a country for table"
    ),
    dcc.Dropdown(
        id='import-export-dropdown',
        options=[
            {'label': 'Import', 'value': 'Import'},
            {'label': 'Export', 'value': 'Export'},
            {'label': 'Both', 'value': 'Both'}
        ],
        placeholder="Select Import, Export, or Both",
        clearable=True
    ),
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': category, 'value': category} for category in df_import_export['Category'].unique()],
        placeholder="Select Category",
        clearable=True
    ),
    dcc.Dropdown(
        id='shipping-method-dropdown',
        options=[{'label': method, 'value': method} for method in df_import_export['Shipping_Method'].unique()],
        placeholder="Select Shipping Method",
        clearable=True
    ),
    dcc.Dropdown(
        id='payment-terms-dropdown',
        options=[{'label': term, 'value': term} for term in df_import_export['Payment_Terms'].unique()],
        placeholder="Select Payment Terms",
        clearable=True
    ),
    dash_table.DataTable(
        id='ports-table',
        columns=[
            {'name': 'Port', 'id': 'Port'},
            {'name': 'Average Quantity', 'id': 'Average_Quantity'},
            {'name': 'Average Value', 'id': 'Average_Value'}
        ],
        page_size=10,
        style_table={'overflowX': 'auto'},
    )
])

# Callbacks
# Callback for time-based import/export graph
@app.callback(
    Output('import-export-graph', 'figure'),
    [Input('import-export-dropdown', 'value'),
     Input('time-period-dropdown', 'value'),
     Input('quantity-value-dropdown', 'value')]
)
def update_graph(import_export_selection, time_period, quantity_value):
    if not import_export_selection or not time_period or not quantity_value:
        return {}
    x_axis = time_period
    filtered_df = df_import_export[df_import_export['Import_Export'].isin(import_export_selection)]
    grouped_df = filtered_df.groupby([x_axis, 'Import_Export'], as_index=False).agg({quantity_value: 'sum'})
    fig = px.bar(grouped_df, x=x_axis, y=quantity_value, color='Import_Export', title=f"{quantity_value} by {time_period}",
                 labels={quantity_value: quantity_value, x_axis: time_period}, color_discrete_map={'Import': 'blue', 'Export': 'orange'})
    return fig

# Callback for country comparison charts
@app.callback(
    [Output('import-export-country-count', 'figure'),
     Output('category-count-country', 'figure'),
     Output('shipping-method-count-country', 'figure')],
    [Input('country-dropdown', 'value'),
     Input('country-import-export-dropdown', 'value')]
)
def update_charts(selected_countries, selected_import_export):
    if not selected_countries or not selected_import_export:
        return {}, {}, {}
    if len(selected_countries) > 3:
        selected_countries = selected_countries[:3]
    if selected_import_export != 'Both':
        filtered_df = df_import_export[(df_import_export['Country'].isin(selected_countries)) &
                                       (df_import_export['Import_Export'] == selected_import_export)]
    else:
        filtered_df = df_import_export[df_import_export['Country'].isin(selected_countries)]
    import_export_fig = px.bar(filtered_df.groupby(['Country', 'Import_Export']).size().reset_index(name='Count'), x='Country', y='Count',
                               color='Import_Export', title='Import/Export Count by Country', barmode='group', color_discrete_sequence=['blue', 'green'])
    category_fig = px.bar(filtered_df.groupby(['Country', 'Category']).size().reset_index(name='Count'), x='Country', y='Count',
                          color='Category', title='Count of Categories by Country', barmode='group', color_discrete_sequence=px.colors.qualitative.Plotly)
    shipping_method_fig = px.bar(filtered_df.groupby(['Country', 'Shipping_Method']).size().reset_index(name='Count'), x='Country', y='Count',
                                 color='Shipping_Method', title='Count of Shipping Methods by Country', barmode='group', color_discrete_sequence=px.colors.qualitative.Plotly)
    return import_export_fig, category_fig, shipping_method_fig

# Callback for port dropdown update
@app.callback(
    Output('port-dropdown', 'options'),
    Input('country-dropdown-chart', 'value')
)
def update_port_dropdown(selected_country):
    if not selected_country:
        return []
    filtered_df = df_import_export[df_import_export['Country'] == selected_country]
    port_options = [{'label': port, 'value': port} for port in filtered_df['Port'].unique()]
    return port_options

# Callback for ports comparison chart
@app.callback(
    Output('import-export-port-bar-chart', 'figure'),
    [Input('country-dropdown-chart', 'value'),
     Input('port-dropdown', 'value')]
)
def update_import_export_port_chart(selected_country, selected_ports):
    if not selected_country or not selected_ports or len(selected_ports) > 2:
        return {}
    filtered_df = df_import_export[(df_import_export['Country'] == selected_country) &
                                   (df_import_export['Port'].isin(selected_ports))]
    fig = px.bar(filtered_df, x='Port', y='Value', color='Import_Export', title='Import and Export Values by Port',
                 barmode='group', color_discrete_sequence=['blue', 'green'])
    return fig

# Callbacks for pie charts
@app.callback(
    Output('shipping-methods-pie-chart', 'figure'),
    [Input('country-dropdown-chart', 'value')]
)
def update_shipping_methods_pie(selected_country):
    if not selected_country:
        return {}
    filtered_df = df_import_export[df_import_export['Country'] == selected_country]
    fig = px.pie(filtered_df, names='Shipping_Method', title='Shipping Methods Distribution')
    return fig

@app.callback(
    Output('payment-terms-pie-chart', 'figure'),
    [Input('country-dropdown-chart', 'value')]
)
def update_payment_terms_pie(selected_country):
    if not selected_country:
        return {}
    filtered_df = df_import_export[df_import_export['Country'] == selected_country]
    fig = px.pie(filtered_df, names='Payment_Terms', title='Payment Terms Distribution')
    return fig

# Callback for ports table
@app.callback(
    Output('ports-table', 'data'),
    [Input('country-dropdown-table', 'value'),
     Input('import-export-dropdown', 'value'),
     Input('category-dropdown', 'value'),
     Input('shipping-method-dropdown', 'value'),
     Input('payment-terms-dropdown', 'value')]
)
def update_ports_table(selected_country, selected_import_export, selected_category, selected_shipping_method, selected_payment_terms):
    filtered_df = df_import_export.copy()
    if selected_country:
        filtered_df = filtered_df[filtered_df['Country'] == selected_country]
    if selected_import_export and selected_import_export != 'Both':
        filtered_df = filtered_df[filtered_df['Import_Export'] == selected_import_export]
    if selected_category:
        filtered_df = filtered_df[filtered_df['Category'] == selected_category]
    if selected_shipping_method:
        filtered_df = filtered_df[filtered_df['Shipping_Method'] == selected_shipping_method]
    if selected_payment_terms:
        filtered_df = filtered_df[filtered_df['Payment_Terms'] == selected_payment_terms]
    grouped_df = filtered_df.groupby('Port').agg(Average_Quantity=('Quantity', 'mean'), Average_Value=('Value', 'mean')).reset_index()
    return grouped_df.to_dict('records')

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


---------------------------------------------------------------------------
DuplicateIdError                          Traceback (most recent call last)
DuplicateIdError: Duplicate component id found in the initial layout: `import-export-dropdown`



<IPython.core.display.Javascript object>